In [55]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import docker
import logging
import time
import concurrent.futures
from datetime import datetime

In [10]:
# Discover the hardware architecture.
avail_cores = os.cpu_count()
print(f"Available cores: {avail_cores}")

Available cores: 48


In [ ]:
# Run CO-Located benchmarks in a Docker container on the same core.
def run_container(task):
    container = task['client'].containers.run(
        image=task['image'],
        command=task['command'],
        cpuset_cpus=task['cpuset_cpus'],
        cgroupns="private",
        detach=True
    )
    print(f"Container started on CPU {task['cpuset_cpus']} for command: {task['command']}")

    # Capture container runtime
    # container_start_time = datetime.strptime(container.attrs['State']['StartedAt'], "%Y-%m-%dT%H:%M:%S.%fZ")
    container_start_time = container.attrs['State']['StartedAt']
    print(f"Container started at {container_start_time}")
    container.stop()
    # container_stop_time = datetime.strptime(container.attrs['State']['FinishedAt'], "%Y-%m-%dT%H:%M:%S.%fZ")
    # print(f"Container stopped at {container_stop_time}")
    # container_lifetime = (container_stop_time - container_start_time).total_seconds()
    # print(f"Container lifetime: {container_lifetime} seconds")
    container.remove()
    print(f"Container on CPU {task['cpuset_cpus']} completed and removed.")

if __name__ == "__main__":
    
    client = docker.from_env()

    # List of dictionaries to hold task information
    tasks = [
        {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=1", "--cpu-max-prime=2000000", "run"], "cpuset_cpus": "1"},
        {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=1M", "--memory-total-size=10G", "--threads=1", "run"], "cpuset_cpus": "2"},
        {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=fileio", "--file-total-size=50G", "--file-test-mode=rndrw", "--num-threads=1", "prepare"], "cpuset_cpus": "3"}, 
        {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=fileio", "cleanup"], "cpuset_cpus": "3"}
    ]
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=3, thread_name_prefix="Colocator") as executor:
        futures = [executor.submit(run_container, task) for task in tasks]
    
    for future in futures:
        future.result()
    print("All isolated benchmarks completed.")
    # Clean up Docker client
    client.close()

In [ ]:
# Run CO-Located benchmarks in a Docker container on the same core.
def run_container(task):
    container = task['client'].containers.run(
        image=task['image'],
        command=task['command'],
        cpuset_cpus=task['cpuset_cpus'],
        cgroupns="private",
        detach=True
    )
    print(f"Container started on CPU {task['cpuset_cpus']} for command: {task['command']}")
    # Capture container statistics
    stats = container.stats(decode=True, stream=True)
    stats_list = []
    for stat in stats:
        stats_list.append(stat) 
    # Stream logs from the container
    for log in container.logs(stream=True, follow=True):
        print(log.strip().decode('utf-8') if isinstance(log, bytes) else log.strip())
    container.stop()
    container.remove()
    print(f"Container on CPU {task['cpuset_cpus']} completed and removed.")

if __name__ == "__main__":
    
    client = docker.from_env()

    # Run every co-located benchmark combination on the same core.
    colocation = [
        {"CpuMem": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=1", "--cpu-max-prime=2000000", "run"], "cpuset_cpus": "1"},
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=1M", "--memory-total-size=10G", "--threads=1", "run"], "cpuset_cpus": "1"}
        ]},
        {"MemFileIO": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=1M", "--memory-total-size=10G", "--threads=1", "run"], "cpuset_cpus": "1"},
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=fileio", "--file-total-size=50G", "--file-test-mode=rndrw", "--num-threads=1", "prepare"], "cpuset_cpus": "1"}
        ]},
        {"FileIOCpu": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=1", "--cpu-max-prime=2000000", "run"], "cpuset_cpus": "1"},
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=fileio", "cleanup"], "cpuset_cpus": "1"}
        ]},
        {"CpuCpu": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=1", "--cpu-max-prime=2000000", "run"], "cpuset_cpus": "1"},
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=cpu", "--num-threads=1", "--cpu-max-prime=2000000", "run"], "cpuset_cpus": "1"},
        ]},
        {"MemMem": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=1M", "--memory-total-size=10G", "--threads=1", "run"], "cpuset_cpus": "1"},
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=memory", "--memory-block-size=1M", "--memory-total-size=10G", "--threads=1", "run"], "cpuset_cpus": "1"},
        ]},
        {"FileIOFileIO": [
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=fileio", "--file-total-size=50G", "--file-test-mode=rndrw", "--num-threads=1", "prepare"], "cpuset_cpus": "1"},
            {"client": client, "image": "niklas/sysbench", "command": ["sysbench", "--test=fileio", "cleanup"], "cpuset_cpus": "1"}
        ]}
    ]
    
    
    with concurrent.futures.ThreadPoolExecutor(max_workers=3, thread_name_prefix="Colocator") as executor:
        futures = [executor.submit(run_container, task) for task in tasks]
    
    for future in futures:
        future.result()
    print("All colocated benchmarks completed.")
    # Clean up Docker client
    client.close()